In [0]:
%sql
CREATE DATABASE IF NOT EXISTS training;

In [0]:
%sql
CREATE OR REPLACE TABLE training.EMP_SCD1_SQL
(
EMPNO INT, 
	ENAME STRING, 
	JOB STRING, 
	MGR INT, 
	HIREDATE DATE, 
	SAL INT, 
	COMM INT, 
	DEPTNO INT
)
USING DELTA
LOCATION '/FileStore/tables/delta-table-merge/EMP_SCD1_SQL';

In [0]:
%sql
select * from training.EMP_SCD1_SQL

EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO


In [0]:
SOURCE_PATH = "dbfs:/FileStore/tables/EMP.csv"
#SOURCE_PATH = "dbfs:/FileStore/tables/EMP_SCD2SQL-1.csv"
df_source = spark.read.options(header=True, delimiter=',', inferSchema='True').csv(SOURCE_PATH)
#spark.read.options(header=True, delimiter=',',inferSchema='True')\
df_source.printSchema()
display(df_source)


root
 |-- EMPNO: integer (nullable = true)
 |-- ENAME: string (nullable = true)
 |-- JOB: string (nullable = true)
 |-- MGR: integer (nullable = true)
 |-- HIREDATE: string (nullable = true)
 |-- SAL: integer (nullable = true)
 |-- COMM: integer (nullable = true)
 |-- DEPTNO: integer (nullable = true)



EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
7839,KING,PRESIDENT,null,17-NOV-81,5000,null,10
7698,BLAKE,MANAGER,7839,01-MAY-81,2850,null,30
7782,CLARK,MANAGER,7839,09-JUN-81,2450,null,10
7566,JONES,MANAGER,7839,02-APR-81,2975,null,20
7788,SCOTT,ANALYST,7566,19-APR-87,3000,null,20
7902,FORD,ANALYST,7566,03-DEC-81,3000,null,20
7369,SMITH,CLERK,7902,17-DEC-80,800,null,20
7499,ALLEN,SALESMAN,7698,20-FEB-81,1600,300,30
7521,WARD,SALESMAN,7698,22-FEB-81,1250,500,30
7654,MARTIN,SALESMAN,7698,28-SEP-81,1250,1400,30


In [0]:
df_source.createOrReplaceTempView('EMP_SOURCE')

In [0]:
%sql
select * from EMP_SOURCE

EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
7839,KING,PRESIDENT,null,17-NOV-81,5000,null,10
7698,BLAKE,MANAGER,7839,01-MAY-81,2850,null,30
7782,CLARK,MANAGER,7839,09-JUN-81,2450,null,10
7566,JONES,MANAGER,7839,02-APR-81,2975,null,20
7788,SCOTT,ANALYST,7566,19-APR-87,3000,null,20
7902,FORD,ANALYST,7566,03-DEC-81,3000,null,20
7369,SMITH,CLERK,7902,17-DEC-80,800,null,20
7499,ALLEN,SALESMAN,7698,20-FEB-81,1600,300,30
7521,WARD,SALESMAN,7698,22-FEB-81,1250,500,30
7654,MARTIN,SALESMAN,7698,28-SEP-81,1250,1400,30


In [0]:
%sql
/*
MERGE INTO training.EMP_SCD1_SQL TARGET
USING EMP_SOURCE SOURCE
ON TARGET.EMPNO = SOURCE.EMPNO
WHEN MATCHED THEN
  UPDATE SET
    TARGET.ENAME = SOURCE.ENAME,
    TARGET.JOB = SOURCE.JOB,
    TARGET.MGR = SOURCE.MGR,
    TARGET.HIREDATE = SOURCE.HIREDATE,
    TARGET.SAL = SOURCE.SAL,
    TARGET.COMM = SOURCE.COMM,
    TARGET.DEPTNO = SOURCE.DEPTNO
WHEN NOT MATCHED THEN
  INSERT
  (
EMPNO,
ENAME,
JOB,
MGR,
HIREDATE,
SAL,
COMM,
DEPTNO
  )
  VALUES
  (
SOURCE.EMPNO,
SOURCE.ENAME,
SOURCE.JOB,
SOURCE.MGR,
SOURCE.HIREDATE,
SOURCE.SAL,
SOURCE.COMM,
SOURCE.DEPTNO
  )
*/

MERGE INTO training.EMP_SCD1_SQL TARGET
USING EMP_SOURCE SOURCE
ON TARGET.EMPNO = SOURCE.EMPNO
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED THEN
  INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
14,0,0,14


In [0]:
%sql
select * from training.EMP_SCD1_SQL

EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
7839,KING,PRESIDENT,null,null,5000,null,10
7698,BLAKE,MANAGER,7839,null,2850,null,30
7782,CLARK,MANAGER,7839,null,2450,null,10
7566,JONES,MANAGER,7839,null,2975,null,20
7788,SCOTT,ANALYST,7566,null,3000,null,20
7902,FORD,ANALYST,7566,null,3000,null,20
7369,SMITH,CLERK,7902,null,800,null,20
7499,ALLEN,SALESMAN,7698,null,1600,300,30
7521,WARD,SALESMAN,7698,null,1250,500,30
7654,MARTIN,SALESMAN,7698,null,1250,1400,30
